# 2) Code Generation with ReACT Prompting

**Goal:** Generate Python code using reasoning before execution (ReACT pattern: Reason → Act → Observe).

**What this notebook shows:**
- A compact ReACT-style loop that creates a plan, emits code, executes it, and validates the result.
- Deterministic behavior (no external API).

### Task
Implement a function `top_k_frequent(nums, k)` that returns the `k` most frequent elements in a list.

We'll craft a reasoning string (the 'Thought') and then an Action that emits Python code.

In [1]:

from typing import Tuple
import textwrap

def react_generate(task: str) -> Tuple[str, str]:
    """
    Returns (thought, code_str) deterministically for the given task.
    """
    thought = textwrap.dedent(f"""
    Thought:
    I will solve "{task}" by counting frequencies with a dictionary, then sorting by count descending.
    I'll return the first k elements' values, not pairs.
    Complexity: O(n log n) due to sorting.
    """).strip()
    code = textwrap.dedent("""
    def top_k_frequent(nums, k):
        from collections import Counter
        counts = Counter(nums)
        # sort by frequency desc, then by value for stable tie-breaks
        pairs = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
        return [v for v,_ in pairs[:k]]
    """).strip()
    return thought, code

TASK = "Return the k most frequent elements of a list."
thought, code_str = react_generate(TASK)

print("=== THOUGHT ===")
print(thought)
print("\n=== GENERATED CODE ===")
print(code_str)

# Execute the generated code in a clean namespace
ns = {}
exec(code_str, ns, ns)

# Quick tests
tests = [
    (([1,1,1,2,2,3], 2), [1,2]),
    ((["a","b","a","c","b","a"], 1), ["a"]),
    (([3,3,2,2,2,1], 2), [2,3]),
]

print("\n=== TESTS ===")
for (args, expected) in tests:
    got = ns["top_k_frequent"](*args)
    print(f"input={args} -> got={got} expected={expected} ✓" if got==expected else f"✗ got={got}, expected={expected}")


=== THOUGHT ===
Thought:
I will solve "Return the k most frequent elements of a list." by counting frequencies with a dictionary, then sorting by count descending.
I'll return the first k elements' values, not pairs.
Complexity: O(n log n) due to sorting.

=== GENERATED CODE ===
def top_k_frequent(nums, k):
    from collections import Counter
    counts = Counter(nums)
    # sort by frequency desc, then by value for stable tie-breaks
    pairs = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
    return [v for v,_ in pairs[:k]]

=== TESTS ===
input=([1, 1, 1, 2, 2, 3], 2) -> got=[1, 2] expected=[1, 2] ✓
input=(['a', 'b', 'a', 'c', 'b', 'a'], 1) -> got=['a'] expected=['a'] ✓
input=([3, 3, 2, 2, 2, 1], 2) -> got=[2, 3] expected=[2, 3] ✓
